In [4]:
%pip install accelerate -U

%pip install torch -U
%pip install transformers -U

pip install pandas scikit-learn numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 1.8 MB/s eta 0:00:00a 0:00:01
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 26.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.4/393.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 25.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.5/167.5

# Train The NLP Model

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

from zipfile import ZipFile
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import accelerate
import transformers

transformers.__version__, accelerate.__version__

zip_path = '' # Provide the Emotion-Lexicon Dataset path name
extract_to = ''  # Provide the Emotion-Lexicon Extraction path name
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

file_path = '/content/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'  # Replace the extraction path name with yours
emotion_lexicon = pd.read_csv(file_path, names=["word", "emotion", "association"], sep='\t')

# Pivot the table to have one row per word and emotion columns
emotion_data = emotion_lexicon.pivot(index="word", columns="emotion", values="association").reset_index()

print(emotion_data.head())

In [ ]:
# Select the required emotion catagories
emotions = ["anger", "anticipation", "disgust", "fear", "joy", "sadness", "surprise", "trust"]
emotion_data_filtered = emotion_data[["word"] + emotions]
print(emotion_data_filtered.head())


In [ ]:
from sklearn.metrics import f1_score, accuracy_score

# Extract labels from the DataFrame
labels = emotion_data_filtered[emotions].values


def compute_metrics(pred):
    # Thresholding at 0.5 to decide on label presence
    preds = pred.predictions >= 0.5
    labels = pred.label_ids

    # Calculate F1 scores
    f1_micro = f1_score(labels, preds, average='micro')
    f1_macro = f1_score(labels, preds, average='macro')

    # Returning multiple metrics
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
    }

class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        # Ensure the lengths match
        assert len(self.encodings['input_ids']) == len(self.labels)

    def __getitem__(self, idx):
        # Ensure idx is within the range
        if idx >= len(self):
            raise IndexError("Index out of range in EmotionDataset")

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize all texts
# Filter NaN rows
emotion_data_filtered = emotion_data_filtered.dropna(subset=['word']).reset_index(drop=True)

labels = emotion_data_filtered[emotions].values

texts = emotion_data_filtered['word'].dropna().astype(str).tolist()
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)

# Ensure the lengths of encodings and labels are the same
assert len(encodings['input_ids']) == len(labels), "The lengths of encodings and labels do not match after filtering."
print(f"Length of encodings: {len(encodings['input_ids'])}")
print(f"Length of labels: {len(labels)}")

# Create the dataset
dataset = EmotionDataset(encodings, labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    logging_dir='./logs',
)

# Check lengths before training
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

try:
    # Try to retrieve the first item as a check
    first_train_item = train_dataset[0]
    first_val_item = val_dataset[0]
except IndexError as e:
    print("An error occurred when retrieving an item from the dataset:", e)

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(emotions))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Define your compute_metrics function for accuracy, etc.
)

trainer.train()

model.save_pretrained('') # Add a save path to save the model


# Emoji Generation Based on Emotion Analysis by NLP Model ⬇️

In [27]:
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
import torch

# The folder path where the model and weights are saved
model_path = '.../V1 Lyrics_to_emoji'

# Load the model
model = BertForSequenceClassification.from_pretrained(model_path)

# Make sure you also use the correct tokenizer, here use the same tokenizer as during training
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



text = '''There's a fire starting in my heart
Reaching a fever pitch, it's bringing me out the dark
Finally I can see you crystal clear
Go ahead and sell me out and I'll lay your ship bare
See how I'll leave with every piece of you
Don't underestimate the things that I will do
There's a fire starting in my heart
Reaching a fever pitch and it's bringing me out the dark
The scars of your love remind me of us
They keep me thinking that we almost had it all
The scars of your love they leave me breathless
I can't help feeling
We could've had it all (you're gonna wish you)
(Never had met me)
Rolling in the deep (tears are gonna fall)
(Rolling in the deep)
You had my heart inside (you're gonna wish you)
Of your hands (never had met me)
And you played it (tears are gonna fall)
To the beat (rolling in the deep)
Baby, I have no story to be told
But I've heard one on you, now I'm gonna make your head burn
Think of me in the depths of your despair
Make a home down there, as mine sure won't be shared
The scars of your love (never had met me)
Remind me of us (tears are gonna fall)
They keep me thinking (rolling in the deep)
That we almost had it all (you're gonna wish you)
The scars of your love (never had met me)
They leave me breathless (tears are gonna fall)
I can't help feeling (rolling in the deep)
We could've had it all (you're gonna wish you)
(Never had met me)
Rolling in the deep (tears are gonna fall)
(Rolling in the deep)
You had my heart inside (you're gonna wish you)
Of your hands (never had met me)
And you played it (tears are gonna fall)
To the beat (rolling in the deep)
We could've had it all
Rolling in the deep
You had my heart inside of your hand
But you played it with a beating
Throw your soul through every open door (whoa)
Count your blessings to find what you look for (whoa)
Turn my sorrow into treasured gold (whoa)
You pay me back in kind and reap just what you've sown
We could've had it all (tears are gonna fall, rolling in the deep)
We could've had it all (you're gonna wish you never had met me)
It all, it all, it all (tears are gonna fall, rolling in the deep)
We could've had it all (you're gonna wish you)
(Never had met me)
Rolling in the deep (tears are gonna fall)
(Rolling in the deep)
You had my heart inside (you're gonna wish you)
Of your hands (never had met me)
And you played it (tears are gonna fall)
To the beat (rolling in the deep)
Could've had it all (you're gonna wish you)
(Never had met me)
Rolling in the deep (tears are gonna fall)
(Rolling in the deep)
You had my heart inside (you're gonna wish you)
Of your hands (never had met me)
But you played it, you played it, you played it
You played it to the beat


'''
inputs = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt")



with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits
predictions = torch.sigmoid(logits).numpy()  # Apply sigmoid and convert to numpy

threshold = 0.5
predicted_labels = (predictions > threshold).astype(int)

emotions_output = ["anger", "anticipation", "disgust", "fear", "joy", "sadness", "surprise", "trust"]

# Print the probability of each emotion dimension
for i, emotion1 in enumerate(emotions_output):
    print(f"{emotion1}: {predictions[0][i].item():.4f}")

predictions_list = []

for i in range(predictions.shape[1]): 
    predictions_list.append(float(f"{predictions[0][i].item():.4f}"))

# Print a list of converted predicted probabilities
print(predictions_list)


anger: 0.0782
anticipation: 0.1479
disgust: 0.0504
fear: 0.1721
joy: 0.0615
sadness: 0.1605
surprise: 0.1051
trust: 0.0163
[0.0782, 0.1479, 0.0504, 0.1721, 0.0615, 0.1605, 0.1051, 0.0163]


In [28]:
# Use the function to generate emoji
emotionVectorToEmoji(predictions_list)

😓: 0.9359966768247223
😟: 0.9329072697488164
😨: 0.9312215434214443
😕: 0.9268771173116092
😳: 0.8997348964279756


# This Is for Emoji Generation Based on Keywords Analysis by NLP Model ⬇️

In [26]:
# given lyrics, create embeddings, search for similar embeddings in the pinecone index, return the emoji unicode
from openai import OpenAI
from pinecone import Pinecone
import os

# convert unicode to emoji
def convert_unicode_to_emoji(unicode):
    return ''.join([chr(int(code.split('+')[1], 16)) for code in unicode.split()])# split the unicode string into a list of codes, convert each code to an integer, and then to a character, and join the characters into a string

# use openai to convert lyrics into text embeddings and search in the pinecone database for similar text
openai_client = OpenAI(
    api_key="", # Provide your OpenAI api key
    base_url="" # Provide your base url
)

lyrics = """
If I should stay
I would only be in your way
So I'll go, but I know
I'll think of you every step of the way
And I will always love you
I will always love you
You
My darling, you, mm, mm
Bittersweet memories
That is all I'm taking with me
So goodbye, please don't cry
We both know I'm not what you, you need
And I will always love you
I will always love you
You
I hope life treats you kind
And I hope you have all you've dreamed of
And I wish you joy and happiness
But above all this, I wish you love
And I will always love you
I will always love you
I will always love you
I will always love you
I will always love you
I, I will always love you
You
Darling, I love you
I'll always, I'll always love you


"""

# use OpenAI API text-embedding-3-small to get embeddings for the lyrics
response = openai_client.embeddings.create(
    input=lyrics,
    model="text-embedding-3-small"
)
lyrics_vector = response.data[0].embedding

# use Pinecone to search for the most relevant papers
pc = Pinecone(api_key="31dd2029-69ce-4016-a5a1-39db8e325fb6")
index_name = "emoji-description"
index = pc.Index(index_name)

# search for the most relevant emojis
query_results = index.query(
    top_k=10, # top 10 most relevant papers
    vector=lyrics_vector,
    includeValues=True,
    includeMetadata=True,
    )

# get the metadata (unicode) of the most relevant emoji
unicode = [item['metadata']['unicode'] for item in query_results['matches']]
final_unicode = []  # remove duplicates
for i in unicode:
    if i not in final_unicode:
        final_unicode.append(i)
    if len(final_unicode) == 5: # only extract top5 emojis
        break

# convert unicode to emoji
final_emoji = [convert_unicode_to_emoji(i) for i in final_unicode]
for i in final_emoji:
    print(f" {i}")

 💌
 🏩
 👩‍❤️‍👩
 👩‍❤️‍👨
 👨‍❤️‍👨


# This is emotion_to_emoji function ⬇️

In [6]:
%pip install openai
%pip install pinecone-client

  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 1.4 MB/s eta 0:00:00ta 0:00:01
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 3.6 MB/s eta 0:00:00a 0:00:01
Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.2/395.2 kB 7.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.0 MB/s eta 0:00:0000:0100:01
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 2.1 MB/s eta 0:00:00a 0:00

In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# load emoji emotion data
df = pd.read_csv('.../EmoTag1200-scores.csv') # Provide the path name of EmoTag1200-scores dataset on your computer
# convert emoji vectors to numpy array
emoji_vectors = df[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']].to_numpy()

# convert emotion vector to emoji based on cosine similarity
def emotionVectorToEmoji(emotionVector):
  emotionVector = np.array(emotionVector).reshape(1, -1)
  # calculate cosine similarity between text_vector and emoji_vectors
  similarities = cosine_similarity(emotionVector, emoji_vectors)
  # combine emoji and similarity into a list
  emoji_similarities = list(zip(df['emoji'], similarities[0]))
  # sort the list by similarity and get the top 10 most similar emojis
  sorted_emojis = sorted(emoji_similarities, key=lambda x: x[1], reverse=True)[:5]

  for emoji, similarity in sorted_emojis:
    print(f"{emoji}: {similarity}")
